# Self Consistency and Multiple paths for reasoning

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

#Multiple Reasoning Paths

In [5]:
from langchain_core.prompts import PromptTemplate

def generate_multiple_paths(problem, num_paths = 3):
    """
    Generate multiple reasoning paths for given problem

    Args:
        problem (str) : the problem statement
        num_paths (int) : Number of reasoning paths to generate.
    
    Returns:
        list: A list of generated reasoning paths
    """

    prompt_template = PromptTemplate(
        input_variables=["problem", "path_num"],
        template= """
            Solve the following problem using a unique approach. This is reasoning path {path_num}
            Problem: {problem}
            Reasoning Path: {path_num}
        """
    )

    paths = []
    for i in range(num_paths):
        chain = prompt_template | llm
        response = chain.invoke({"problem": problem, "path_num": i+1}).content
        paths.append(response)

    return paths

In [6]:
problem = "A ball is thrown upwards with an initial velocity of 20 m/s. How high will it go?"
paths = generate_multiple_paths(problem)

for i, path in enumerate(paths, 1):
    print(f"Path {i}:\n{path}\n")

Path 1:
A classic problem!

Reasoning Path 1: Let's think about the ball's motion in a way that's a bit unconventional. Instead of using traditional physics equations, let's consider the ball's velocity as it rises.

When the ball is thrown upwards, its initial velocity is 20 m/s. As it rises, the ball's velocity decreases due to the force of gravity (which is -9.8 m/s^2). But here's the key insight: the ball's velocity is not constant; it's changing all the time.

Imagine the ball's velocity as a line on a graph, with time on the x-axis and velocity on the y-axis. The line would start at 20 m/s and gradually decrease as the ball rises. At some point, the ball's velocity will reach zero, which means it's momentarily at rest. This is the highest point it will reach.

Now, think of the ball's height as the area under the velocity-time graph. As the ball rises, its height increases, and when its velocity reaches zero, its height is at its maximum.

By integrating the velocity-time graph (

#Aggrigating Results

In [7]:
def aggregate_results(paths):
    """
    Aggregate results from various reasoning paths.

    Args:
        paths(list): List of Reasoning paths

    Returns:
        str: The most consistent answer.
    """
    prompt_template = PromptTemplate(
        input_variables=["paths"],
        template="""
            Analyze the following reasoning paths and determine the most consistent answer.
            If there are descripencies, explain why and provide the most likely correct answer.
            Reasoning paths:
            {paths}
            Most Consistent Answer:
        """
    )

    chain = prompt_template | llm
    response = chain.invoke({"paths": "\n".join(paths)}).content
    return response

In [8]:
aggregated_result = aggregate_results(paths)
print("Aggregated Result:\n", aggregated_result)

Aggregated Result:
 The most consistent answer is **Reasoning Path 2: Analyze the problem from a different perspective**, which estimates the maximum height to be approximately **20.4 meters**.

Here's why:

* The other two reasoning paths arrive at different answers:
	+ Reasoning Path 1 uses an unconventional approach and estimates the maximum height to be approximately **20 meters**.
	+ Reasoning Path 3 uses the energy conservation principle and estimates the maximum height to be approximately **10.2 meters**.
* Reasoning Path 2 provides a more accurate and consistent answer. The equation of motion is correctly applied to relate the initial velocity, height, and potential energy. The calculation is also straightforward and easy to follow.
* The other two reasoning paths have some inconsistencies or oversimplifications:
	+ Reasoning Path 1 uses a rough approximation and assumes the average velocity over a small time interval, which might not accurately capture the ball's motion.
	+ Re

#Self-Consistency Check

In [9]:
def self_consistency_check(problem, aggregated_result):
    """
    Perform a self-consistency check on the aggregated result.

    Args:
        problem (str): The original problem statement.
        aggregated_result (str): The aggregated result to check.

    Returns:
        str: An evaluation of the result's consistency and reliability.
    """
    prompt_prompt = PromptTemplate(
        input_variables=["problem", "result"],
        template="""
            Evaluate the consistency and reliability of following  result to the given problem.
            Problem: {problem}
            Result: {result}

            Evaluation (consider factors like logical consisteny, adhere to known facts, and potential biases):
        """
    )
    chain = prompt_prompt | llm
    response = chain.invoke({"problem": problem, "result": aggregated_result}).content
    return response

In [10]:
consistency_evaluation = self_consistency_check(problem, aggregated_result)
print("Self-Consistency Evaluation:\n", consistency_evaluation)

Self-Consistency Evaluation:
 Here's the evaluation of the consistency and reliability of the result:

**Logical Consistency:**

* The result is based on the application of a well-established concept (equation of motion) to relate the initial velocity, height, and potential energy.
* The calculation is straightforward and easy to follow, making it easy to understand and reproduce the result.
* The conclusion is consistent with the initial conditions and assumptions made in the problem.

**Adherence to Known Facts:**

* The problem is based on a fundamental concept in physics (motion under gravity), and the result is consistent with our understanding of this concept.
* The use of the equation of motion is a standard approach to solve problems involving objects under the influence of gravity.
* The result is not in conflict with any known scientific facts or principles.

**Potential Biases:**

* The evaluation seems to be based on a fair and objective assessment of the different reasonin

#Problems

In [11]:
def solve_problem(problem):
    """
    Solve a problem using multiple reasoning paths, aggregation, and self-consistency check.

    Args:
    problem (str): The problem statement.

    Returns:
    tuple: (aggregated_result, consistency_evaluation)
    """

    paths = generate_multiple_paths(problem)
    aggregated_result = aggregate_results(paths)
    consistency_evaluation = self_consistency_check(problem, aggregated_result)
    return aggregated_result, consistency_evaluation

In [12]:
problems = [
    "What is the capital of France?",
    "Explain the concept of supply and demand in economics.",
    "If a train travels at 60 km/h, how long will it take to cover 180 km?"
]

In [13]:
for problem in problems:
    print(f"Problem: {problem}")
    result, evaluation = solve_problem(problem)
    print("Aggregated Result:\n", result)
    print("\nConsistency Evaluation:\n", evaluation)
    print("\n" + "-"*50 + "\n")

Problem: What is the capital of France?
Aggregated Result:
 The most consistent answer is **Paris**. Each of the three reasoning paths arrives at the same conclusion: Paris is the capital of France.

Reasoning Path 1 uses a combination of historical and linguistic reasoning, connecting the French Crown Jewels, the Louvre Museum, and the French language to arrive at Paris as the capital.

Reasoning Path 2 takes a more creative approach, using the Eiffel Tower as a metaphorical guide to lead to the capital, with a series of subtle clues and landmarks along the way, ultimately ending at Paris.

Reasoning Path 3 uses a more unconventional approach, associating Claude Monet's painting "Impression, Sunrise" with the city of Le Havre, and then contrasting Le Havre with Paris in terms of geography and culture to arrive at the conclusion that Paris is the capital.

All three paths are creative and unique, but they ultimately converge on the same answer: Paris is the capital of France.

Consiste